In [1]:
import pandas as pd
import time
import numpy as np
from urllib.error import HTTPError

## Datasheet creation

First we used the API of Stemspy to get all games and there information. This was done using the ```pd.read_json()``` while using the API url as input. Because we didn't know how many games there where, we tried the api call until it returns a HTTPError. 

In [66]:
url = "https://steamspy.com/api.php?request=all&page="

ls = []
for i in range(80):
  try:
    data = pd.read_json(url+f"{i}").T
    ls.append(data)
    time.sleep(60)
  except HTTPError:
    break
df = pd.concat(ls, axis=0, ignore_index=True)

After generating the first part of our dataset, we saved it using the pandas function ```to_pickle()```. 

In [68]:
df.to_pickle("dataset.pkl")

After saving the first part, we had to get the detailed information for every instance. This was also done with the Steamspy API. 

In [57]:
df = pd.read_pickle("dataset.pkl")

detailed_url="https://steamspy.com/api.php?request=appdetails&appid="

pickl_df = pd.read_pickle("detailed.pkl")

ls = []
for idx, row in pickl_df.iterrows():
  ls.append(row)

To achieve that, we iterrate over the datset of part one and request the detailed information. Because of some default formating of ```pd.read_json()``` we had to construct a new pandas DataFrame to match our expectations. 

In [58]:
i = 0
for idx, rows in df.iterrows():
  temp_df = pd.read_json(detailed_url+f"{rows['appid']}")
  if temp_df.empty:
    continue
  detailed_df = pd.DataFrame([[temp_df.index.values],[temp_df.genre.values],[temp_df.languages.values]], 
             columns=[temp_df.appid[0]], 
             index=["tags","genre","language"]).T
  ls.append(detailed_df)
  time.sleep(1)
  i = i+1
  if i >= 20:
     df_detailed = pd.concat(ls, axis=0, ignore_index=False)
     df_detailed.to_pickle("detailed.pkl")
     i = 0

df_detailed = pd.concat(ls, axis=0, ignore_index=False)

After that we only need to combine the two parts into one dataset and save it to disk. 

In [ ]:
df_complete = pd.merge(df,df_detailed, left_index=True, right_index=True)

df_complete.to_pickle("complete_df_steam.pkl")